# 2.2.2 Planzeichnung OCR – (GPT4o)

* Zero-Shot
* Chain-Of-Thought

In [1]:
from utils.parser import Parser
from utils.openai import OpenAI
import asyncio

instructions = "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren. Achte auf Vollständigkeit."
ava = OpenAI(instructions)
parser = Parser()

In [20]:
ava.clearContext()
ava.addContext([
    # Output_1_PZE (best-case-scenario)
    {
        "type": "text",
        "text": "Hier ist die extrahierte Struktur der Nutzungsschablone in ASCII-Tabellenform: +---------------------+---------------------+| Art der Nutzung     | Gebäudehöhe         |+---------------------+---------------------+| Grundflächenzahl    | Nutzungsschablone   || Geschossflächenzahl | (§ 9 (1) 1 BauGB)   |+---------------------+---------------------+| Bauweise            |                     |+---------------------+---------------------+"
    },
    # Output_2_PZE (best-case-scenario)
    # {
    #     "type": "text",
    #     "text": 'Erklärung der Begriffe der Nutzungsschablone im JSON-Format: {"Art der Nutzung": {"Definition": "Gibt die Art der Nutzung des Grundstücks an, z.B. Gewerbegebiet.","Beispiel": "GE (Gewerbegebiet)"},"Gebäudehöhe": {"Definition": "Maximal zulässige Gebäudehöhe über Erdgeschossfußbodenhöhe.","Beispiel": "GH=max.12,00"},"Grundflächenzahl": {"Definition": "Gibt das Verhältnis der bebauten Fläche zur Grundstücksfläche an.","Beispiel": "0,6"},"Geschossflächenzahl": {"Definition": "Gibt das Verhältnis der gesamten Geschossfläche zur Grundstücksfläche an.","Beispiel": "1,2"},"Bauweise": {"Definition": "Gibt die Art und Weise der Bebauung an, z.B. abweichende Bauweise.","Beispiel": "a (abweichende Bauweise)"}}'
    # },
])

In [17]:
print(ava.aiMessage)

content=[{'type': 'text', 'text': 'Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren. Achte auf Vollständigkeit.'}, {'type': 'text', 'text': 'Hier ist die extrahierte Struktur der Nutzungsschablone in ASCII-Tabellenform: +---------------------+---------------------+| Art der Nutzung     | Gebäudehöhe         |+---------------------+---------------------+| Grundflächenzahl    | Nutzungsschablone   || Geschossflächenzahl | (§ 9 (1) 1 BauGB)   |+---------------------+---------------------+| Bauweise            |                     |+---------------------+---------------------+'}, {'type': 'text', 'text': 'Erklärung der Begriffe der Nutzungsschablone im JSON-Format: {"Art der Nutzung": {"Definition": "Gibt die Art der Nutzung des Grundstücks an, z.B. Gewerbegebiet.","Beispiel": "GE (Gewerbegebiet)"},"Gebäudehöhe": {"Definition": "Maximal zulässige Gebäudehöhe über Erdgeschossfußbodenhöhe.","Beispiel": "GH=max.12,00"},"Grundflächenzahl":

In [18]:
# Planzeichnung: Nutzungsschablone
# pz_raw_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-PZ-raw.png"
pz_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-PZ.png"
pz_crop_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-PZ-crop.png"

async def run(pz_path):
    base64_pz, img_size = parser.imageToBase64(pz_path)
    print("image_size:", img_size)
    message = ava.request([
        {
            "type": "text",
            "text": 'Gehe Schritt für Schritt vor: 1. Finde die Nutzungsschablonen in der vorliegenden Planzeichnung. 2. Gebe die Nutzungsschablone jeweils im JSON-Format aus.'
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_pz}",
                "detail": "high"
            }
        },
    ])
    return message

messages = await asyncio.gather(run(pz_path), run(pz_crop_path))
for message in messages:
    print(message)
    print("##############################################")

image_size: [1040, 768]
image_size: [1359, 768]
Schritt 1: Finde die Nutzungsschablonen in der vorliegenden Planzeichnung.

In der Planzeichnung sind mehrere Nutzungsschablonen zu erkennen. Diese sind durch die Beschriftungen "GE", "GH=max.12,00", "0,6", "1,2" und "a" gekennzeichnet.

Schritt 2: Gebe die Nutzungsschablone jeweils im JSON-Format aus.

Hier sind die Nutzungsschablonen im JSON-Format:

1. Nutzungsschablone (links oben):
```json
{
  "Art der Nutzung": "GE (Gewerbegebiet)",
  "Gebäudehöhe": "GH=max.12,00",
  "Grundflächenzahl": "0,6",
  "Geschossflächenzahl": "1,2",
  "Bauweise": "a (abweichende Bauweise)"
}
```

2. Nutzungsschablone (rechts oben):
```json
{
  "Art der Nutzung": "GE (Gewerbegebiet)",
  "Gebäudehöhe": "GH=max.12,00",
  "Grundflächenzahl": "0,6",
  "Geschossflächenzahl": "1,2",
  "Bauweise": "a (abweichende Bauweise)"
}
```

3. Nutzungsschablone (rechts unten):
```json
{
  "Art der Nutzung": "GE (Gewerbegebiet)",
  "Gebäudehöhe": "GH=max.12,00",
  "Grundfläch

In [6]:
# Planzeichnung: Nutzungsschablone – BOUNDING BOX
pze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-PZ-bb-crop.png"
base64_pz, img_size = parser.imageToBase64(pz_path)
print("image_size:", img_size)

message = ava.request([
    {
        "type": "text",
        "text": 'Gehe Schritt für Schritt vor: 1. Finde alle rot umrandeten Nutzungsschablonen in der vorliegenden Planzeichnung. 2. Gebe die Nutzungsschablonen jeweils im JSON-Format aus: {<Parameter>:<Wert>}'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_pz}",
            "detail": "high"
        }
    },
])

print(message)

image_size: [1040, 768]
Hier sind die rot umrandeten Nutzungsschablonen aus der Planzeichnung im JSON-Format:

1. Nutzungsschablone 1:
```json
{
  "Art der Nutzung": "GE",
  "Gebäudehöhe": "GH=max.12,00",
  "Grundflächenzahl": "0,6",
  "Geschossflächenzahl": "1,2",
  "Bauweise": "a"
}
```

2. Nutzungsschablone 2:
```json
{
  "Art der Nutzung": "GE",
  "Gebäudehöhe": "GH=max.12,00",
  "Grundflächenzahl": "0,6",
  "Geschossflächenzahl": "1,2",
  "Bauweise": "a"
}
```

3. Nutzungsschablone 3:
```json
{
  "Art der Nutzung": "GE",
  "Gebäudehöhe": "GH=max.12,00",
  "Grundflächenzahl": "0,6",
  "Geschossflächenzahl": "1,2",
  "Bauweise": "a"
}
```

4. Nutzungsschablone 4:
```json
{
  "Art der Nutzung": "GE",
  "Gebäudehöhe": "GH=max.12,00",
  "Grundflächenzahl": "0,6",
  "Geschossflächenzahl": "1,2",
  "Bauweise": "a"
}
```

Alle rot umrandeten Nutzungsschablonen in der Planzeichnung haben die gleichen Parameterwerte.


In [21]:
# Planzeichnung: Nutzungsschablone – VISUAL SEARCH + CROPPING
image_url_prompts = []
for index in range(4):
    base64_nz, img_size = parser.imageToBase64(f'../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-PZ-nz{index+1}-crop.png')
    image_url_prompts.append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_nz}",
            "detail": "high"
        }
    })

message = ava.request([
    *image_url_prompts,
    {
        "type": "text",
        "text": 'Gebe die Werte der jeweiligen Nutzungsschablonen strukturiert im JSON-Format aus: {<Parameter>:<Wert>}'
    }
])

print(message)

Hier sind die Werte der jeweiligen Nutzungsschablonen strukturiert im JSON-Format:

1. Bild:
```json
{
  "Art der Nutzung": "GEE*",
  "Gebäudehöhe": "max. 12,00",
  "Grundflächenzahl": "0,6",
  "Geschossflächenzahl": "1,2",
  "Bauweise": "a"
}
```

2. Bild:
```json
{
  "Art der Nutzung": "GEE",
  "Gebäudehöhe": "max. 12,00",
  "Grundflächenzahl": "0,6",
  "Geschossflächenzahl": "1,2",
  "Bauweise": "a"
}
```

3. Bild:
```json
{
  "Art der Nutzung": "GE",
  "Gebäudehöhe": "max. 12,00",
  "Grundflächenzahl": "0,6",
  "Geschossflächenzahl": "1,2",
  "Bauweise": "a"
}
```

4. Bild:
```json
{
  "Art der Nutzung": "GE",
  "Gebäudehöhe": "max. 12,00",
  "Grundflächenzahl": "0,6",
  "Geschossflächenzahl": "1,2",
  "Bauweise": "a"
}
```
